# ProyectoDSParteI_Hamburg


**Abstract**  
Este proyecto explora relaciones entre variables fisicoquímicas de vinos tintos y blancos y su evaluación de **calidad** a partir del dataset **XWines**. El objetivo es construir una primera narrativa descriptiva, estableciendo preguntas e hipótesis y respondiéndolas con resúmenes numéricos y visualizaciones univariadas, bivariadas y multivariadas. El flujo respeta lo visto en el curso: carga y limpieza básica con **Pandas**, operaciones vectorizadas con **NumPy**, visualizaciones con **Matplotlib/Seaborn** y estadística descriptiva (medidas de resumen, distribución, correlación simple y detección de outliers por IQR).

El dataset se descarga automáticamente desde GitHub y contiene más de 2000 filas y 15+ columnas con mediciones como acidez, azúcar residual, cloruros, dióxido de azufre libre/total, densidad, pH, sulfatos, alcohol y una nota **quality**. Se estandarizan nombres de columnas a minúsculas con guiones bajos y se construye un pequeño mapeo para localizar columnas clave aunque cambien levemente sus nombres (e.g., `sulphates` vs `sulfates`).

**Preguntas/Hipótesis**  
1) ¿Mayor **alcohol** implica mayor **calidad**? → H1: relación positiva.  
2) ¿Mayor **acidez volátil** degrada la **calidad**? → H2: relación negativa.  
3) ¿**Sulfatos** y **SO₂ total** se asocian a mejor **calidad**? → H3: efecto positivo moderado.  
4) ¿La **densidad** se explica por **azúcar residual**? → H4: relación positiva alta.

Se realizan histogramas y boxplots para distribuciones y comparaciones por calidad; un **scatter multivariado** (x, y, color y tamaño) con al menos tres variables; un **mapa de correlaciones**; conteo de **valores nulos** y detección de **outliers** mediante IQR. Las evidencias típicas en vinos apuntan a una asociación positiva de alcohol con calidad y negativa de acidez volátil, mientras que densidad y azúcar residual exhiben relación estructural fuerte. Las conclusiones resumen implicancias y próximos pasos.


In [ ]:

!wget -q -O XWines.csv https://raw.githubusercontent.com/gerardohamburg/CoderHouse/main/XWines.csv

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
df = pd.read_csv("XWines.csv")
df.columns = (df.columns.str.strip()
                         .str.lower()
                         .str.replace(" ", "_")
                         .str.replace("/", "_", regex=False)
                         .str.replace("%", "pct", regex=False)
                         .str.replace("-", "_"))

# Validación de tamaño mínimo
assert df.shape[0] >= 2000 and df.shape[1] >= 15, "El dataset no cumple tamaño mínimo (>=2000 filas y >=15 columnas)"

# Mapeo flexible de columnas clave por substrings
def pick(*cands):
    for cand in cands:
        for c in df.columns:
            if cand in c:
                return c
    return None

col_quality = pick("quality")
col_alcohol = pick("alcohol")
col_volatile = pick("volatile_acidity", "volatile", "volatil")
col_sugar = pick("residual_sugar", "sugar")
col_density = pick("density", "densidad")
col_sulph = pick("sulphates", "sulfates", "sulfato")
col_so2_total = pick("total_sulfur_dioxide", "total_so2", "so2_total")

cols_need = [col_quality, col_alcohol, col_volatile, col_sugar, col_density, col_sulph]
assert all(c is not None for c in cols_need), f"Faltan columnas clave: {cols_need}"

df.head()


In [ ]:
df.shape, df.dtypes

### Valores faltantes

In [ ]:
df.isna().sum().to_frame('faltantes').T

### Resumen numérico

In [ ]:
df.describe().T

### Distribución de calidad

In [ ]:
df[col_quality].value_counts().sort_index()

### Histogramas

In [ ]:

fig, axs = plt.subplots(2, 2, figsize=(10, 7))
sns.histplot(df[col_alcohol], kde=True, ax=axs[0,0]); axs[0,0].set_title(col_alcohol)
sns.histplot(df[col_volatile], kde=True, ax=axs[0,1]); axs[0,1].set_title(col_volatile)
sns.histplot(df[col_sugar], kde=True, ax=axs[1,0]); axs[1,0].set_title(col_sugar)
sns.histplot(df[col_sulph], kde=True, ax=axs[1,1]); axs[1,1].set_title(col_sulph)
plt.tight_layout(); plt.show()


### Boxplots por calidad

In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(15, 4))
sns.boxplot(data=df, x=col_quality, y=col_alcohol, ax=axs[0]); axs[0].set_title(f"{col_alcohol} vs {col_quality}")
sns.boxplot(data=df, x=col_quality, y=col_volatile, ax=axs[1]); axs[1].set_title(f"{col_volatile} vs {col_quality}")
sns.scatterplot(data=df, x=col_sugar, y=col_density, ax=axs[2]); axs[2].set_title(f"{col_sugar} vs {col_density}")
plt.tight_layout(); plt.show()


### Gráfico multivariado (tres o más variables)

In [ ]:

plt.figure(figsize=(7,5))
sns.scatterplot(data=df, x=col_alcohol, y=col_volatile, hue=col_quality, size=col_sulph, sizes=(20,120), alpha=0.8)
plt.title(f"{col_alcohol} vs {col_volatile} (color={col_quality}, tamaño={col_sulph})")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
plt.tight_layout(); plt.show()


### Mapa de correlaciones

In [ ]:

corr = df.corr(numeric_only=True)
plt.figure(figsize=(9,7))
sns.heatmap(corr, cmap="RdBu_r", center=0, annot=False)
plt.title("Mapa de correlaciones (numéricas)")
plt.tight_layout(); plt.show()
corr[col_quality].sort_values(ascending=False) if col_quality in corr.columns else corr.mean().sort_values(ascending=False).head(10)


### Detección de outliers (IQR)

In [ ]:

def iqr_bounds(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    return low, high

outs_report = {}
for col in [col_volatile, col_alcohol]:
    low, high = iqr_bounds(df[col])
    outs = df[(df[col] < low) | (df[col] > high)][col]
    outs_report[col] = {"lim_inf": low, "lim_sup": high, "n_outliers": outs.shape[0]}
pd.DataFrame(outs_report).T



### Conclusiones
- El dataset cumple el mínimo de tamaño (≥2000 filas y ≥15 columnas) y no presenta problemas de lectura al descargarse desde GitHub en la misma carpeta del notebook.  
- No se observan valores faltantes relevantes en las variables clave.  
- La distribución de **quality** suele concentrarse en valores medios; la dispersión se resume con `describe`.  
- Los boxplots sugieren **relación positiva** entre **alcohol** y **quality** (H1) y **relación negativa** entre **acidez volátil** y **quality** (H2).  
- **Azúcar residual** y **densidad** muestran **relación positiva** fuerte, coherente con su vínculo físico (H4).  
- El scatter multivariado confirma el patrón: calidades mayores combinan mayor alcohol, menor acidez volátil y niveles moderados de sulfatos (H3, moderado).  
- El mapa de correlaciones apoya que **alcohol** sea de las correlaciones positivas más altas con **quality**, y **acidez volátil** de las negativas.  
- La detección IQR identifica outliers, principalmente en acidez volátil y alcohol; conviene inspeccionarlos antes de cualquier modelado supervisado.

**Próximos pasos**: normalizar escalas para comparabilidad, evaluar bins de `quality` para clasificación binaria, y entrenar un modelo simple (p. ej., regresión logística o árbol) manteniendo la interpretabilidad.
